In [31]:
import random
import copy

### Клас Матриці

In [32]:
class Matrix:
    def __init__(self, data):
        """Ініціалізація матриці списком списків."""
        self.data = copy.deepcopy(data)
        self.rows = len(data)
        self.cols = len(data[0])

    def __str__(self):
        return '\n'.join(['\t'.join([f"{x:.4f}" for x in row]) for row in self.data])

    def __getitem__(self, index):
        return self.data[index]
    
    def __setitem__(self, index, value):
        self.data[index] = value

    # Операція додавання
    def __add__(self, other):
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Розміри матриць не співпадають")
        new_data = [[self[i][j] + other[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(new_data)

    # Операція віднімання
    def __sub__(self, other):
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Розміри матриць не співпадають")
        new_data = [[self[i][j] - other[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(new_data)

    # Множення матриць
    def __mul__(self, other):
        # Якщо other - число
        if isinstance(other, (int, float)):
            new_data = [[self[i][j] * other for j in range(self.cols)] for i in range(self.rows)]
            return Matrix(new_data)
        
        # Якщо other - Вектор (вважаємо його стовпцем) або Матриця
        if isinstance(other, Matrix):
            if self.cols != other.rows:
                raise ValueError(f"Не можна помножити ({self.rows}x{self.cols}) на ({other.rows}x{other.cols})")
            
            result = [[0] * other.cols for _ in range(self.rows)]
            for i in range(self.rows):
                for j in range(other.cols):
                    for k in range(self.cols):
                        result[i][j] += self[i][k] * other[k][j]
            return Matrix(result)

### Алгоритм LUP-розкладу

In [33]:
def lup_decomposition(A):
    """
    Повертає P, L, U такі, що PA = LU.
    A - об'єкт класу Matrix.
    Повертає: (P (список перестановок), L, U)
    """
    n = A.rows
    # Копіюємо A, щоб не зіпсувати оригінал, це буде наша U (і частково L)
    C = copy.deepcopy(A)
    # Вектор перестановок
    P = list(range(n))

    for i in range(n):
        # Опорний елемент (pivot)
        pivot = 0
        pivot_row = -1
        for row in range(i, n):
            if abs(C[row][i]) > pivot:
                pivot = abs(C[row][i])
                pivot_row = row
        
        if pivot == 0:
            raise ValueError("Матриця вироджена (singular), розв'язку не існує.")

        # Міняємо місцями рядки в C та в P
        C.data[i], C.data[pivot_row] = C.data[pivot_row], C.data[i]
        P[i], P[pivot_row] = P[pivot_row], P[i]

        # Оновлення матриці (формування L та U)
        for j in range(i + 1, n):
            C[j][i] /= C[i][i] # Це елемент L
            for k in range(i + 1, n):
                C[j][k] -= C[j][i] * C[i][k] # Це елементи Schur complement

    # Формування окремих матриць L та U для зручності
    L_data = [[0] * n for _ in range(n)]
    U_data = [[0] * n for _ in range(n)]
    
    for i in range(n):
        L_data[i][i] = 1.0
        for j in range(n):
            if j < i:
                L_data[i][j] = C[i][j]
            else:
                U_data[i][j] = C[i][j]

    return P, Matrix(L_data), Matrix(U_data)

### Розв'язання системи LUP

In [34]:
def lup_solve(A, b):
    """
    Розв'язує Ax = b використовуючи LUP.
    A - Matrix (NxN)
    b - список або Matrix (Nx1)
    """
    n = A.rows
    P, L, U = lup_decomposition(A)
    
    # 1. Застосовуємо перестановки до b: Pb
    pb = [0] * n
    # Якщо b це Matrix об'єкт
    b_vals = [b[i][0] for i in range(n)] if isinstance(b, Matrix) else b
    
    for i in range(n):
        pb[i] = b_vals[P[i]]

    # 2. Forward Substitution: Ly = Pb
    y = [0] * n
    for i in range(n):
        s = sum(L[i][j] * y[j] for j in range(i))
        y[i] = pb[i] - s

    # 3. Backward Substitution: Ux = y
    x = [0] * n
    for i in range(n - 1, -1, -1):
        s = sum(U[i][j] * x[j] for j in range(i + 1, n))
        x[i] = (y[i] - s) / U[i][i]
        
    # Повертаємо результат як Матрицю-стовпець
    return Matrix([[val] for val in x])

### 4. Демонстрація (Завдання 4-6)

In [35]:
def generate_system(n):
    """Генерує випадкову систему."""
    # Генеруємо A
    A_data = [[random.randint(-10, 10) for _ in range(n)] for _ in range(n)]
    # Щоб уникнути ділення на 0, додамо діагональну домінантність (не обов'язково, але корисно для рандому)
    for i in range(n):
        if A_data[i][i] == 0: A_data[i][i] = 1
            
    # Генеруємо b
    b_data = [[random.randint(-20, 20)] for _ in range(n)]
    
    return Matrix(A_data), Matrix(b_data)

In [37]:
# Завдання 4: Знайти (генерувати) систему 10x10
N = 10
print(f"\n1. Генерируємо систему рівнянь {N}x{N}...")
A, b = generate_system(N)

print(f"\n--- Повна Матриця A ({N}x{N}) ---")
print(A)
    
print(f"\n--- Повний Вектор b ({N}x1) ---")
print(b)

# Завдання 5: Розв'язати систему
print(f"\n2. Розв'язуємо систему методом LUP...")
x = lup_solve(A, b)

print(f"\n--- Знайдений розв'язок x ({N}x1) ---")
print(x)

# Завдання 6: Перевірка
print("\n3. Перевірка (обчислюємо похибку |Ax - b|):")
b_calc = A * x

error = 0
for i in range(N):
    diff = abs(b_calc[i][0] - b[i][0])
    error += diff

print(f"Сумарна похибка: {error:.2e}")


1. Генерируємо систему рівнянь 10x10...

--- Повна Матриця A (10x10) ---
8.0000	-2.0000	0.0000	-2.0000	-6.0000	-9.0000	7.0000	7.0000	7.0000	1.0000
8.0000	7.0000	-4.0000	1.0000	-2.0000	-9.0000	-2.0000	5.0000	4.0000	-5.0000
3.0000	8.0000	-1.0000	-2.0000	-5.0000	8.0000	0.0000	-2.0000	2.0000	3.0000
2.0000	-8.0000	-10.0000	4.0000	-9.0000	8.0000	6.0000	1.0000	6.0000	6.0000
5.0000	-4.0000	-4.0000	-2.0000	3.0000	-2.0000	1.0000	10.0000	-4.0000	-7.0000
3.0000	-4.0000	-7.0000	-4.0000	-7.0000	-8.0000	1.0000	-5.0000	1.0000	-1.0000
-1.0000	0.0000	3.0000	5.0000	2.0000	-7.0000	-1.0000	-5.0000	-4.0000	-4.0000
1.0000	-4.0000	0.0000	0.0000	-2.0000	8.0000	-1.0000	4.0000	-6.0000	5.0000
-4.0000	-4.0000	-4.0000	3.0000	6.0000	9.0000	-8.0000	-3.0000	3.0000	2.0000
-7.0000	-2.0000	-6.0000	9.0000	9.0000	9.0000	6.0000	2.0000	-2.0000	-8.0000

--- Повний Вектор b (10x1) ---
-15.0000
13.0000
17.0000
-17.0000
-18.0000
17.0000
20.0000
7.0000
-18.0000
0.0000

2. Розв'язуємо систему методом LUP...

--- Знайдений розв'яз

# Звіт

## 1\. Постановка завдання

Метою роботи є опанування методів ефективного розв'язання систем лінійних рівнянь (СЛАР). Необхідно:

1.  Реалізувати класи `Matrix` та `Vector` з базовими арифметичними операціями.
2.  Реалізувати алгоритм **LUP-розкладу** матриці ($PA = LU$).
3.  Застосувати цей алгоритм для розв'язання СЛАР розмірністю не менше $10 \times 10$.
4.  Перевірити правильність отриманого розв'язку.

## 2\. Опис реалізації

### Клас Matrix

Було розроблено клас `Matrix`, який зберігає дані у вигляді списку списків (`list of lists`).
Реалізовані методи:

  * `__add__`, `__sub__`: поелементне додавання та віднімання.
  * `__mul__`: множення матриць (алгоритм "рядок на стовпець", складність $O(N^3)$).
  * Перевантаження операторів дозволяє писати математично зрозумілий код: `C = A * B`.

### LUP-розклад

LUP-розклад представляє матрицю $A$ як добуток матриці перестановок $P$, нижньої трикутної матриці $L$ (з одиницями на діагоналі) та верхньої трикутної матриці $U$.

### Алгоритм розв'язання

Система $Ax = b$ розв'язується у три етапи:

1.  **LUP-розклад:** Знаходимо $P, L, U$.
2.  **Пряма підстановка (Forward Substitution):** Розв'язуємо $Ly = Pb$ відносно $y$.
3.  **Зворотна підстановка (Backward Substitution):** Розв'язуємо $Ux = y$ відносно $x$.

Складність розв'язання: $O(N^3)$ на етапі розкладу та $O(N^2)$ на етапі підстановок.

## 3\. Результати експерименту

Для тестування було згенеровано випадкову систему лінійних рівнянь розмірністю $10 \times 10$ з цілочисельними коефіцієнтами в діапазоні $[-10, 10]$.

**Перевірка:**
Для верифікації було виконано множення вихідної матриці $A$ на знайдений вектор $x$ та порівняння результату з вектором $b$.
Сумарна похибка ($\sum |(Ax)_i - b_i|$) склала менше ніж $10^{-10}$, що свідчить про коректність реалізації та точність методу.

## 4\. Висновки

У ході роботи було створено бібліотеку для роботи з матрицями та реалізовано метод LUP-розкладу.

  * Метод LUP є універсальним для квадратних невироджених матриць.
  * Використання вибору головного елемента (pivoting) забезпечує чисельну стійкість алгоритму.
  * Реалізація успішно справляється з системами розмірністю $10 \times 10$ та більше, забезпечуючи високу точність розв'язку.